<a href="https://colab.research.google.com/github/yiyc-kr/llm-study/blob/main/code-llama-finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://github.com/ragntune/code-llama-finetune/blob/main/fine-tune-code-llama.ipynb

In [2]:
!pip install git+https://github.com/huggingface/transformers.git@main bitsandbytes accelerate==0.20.3  # we need latest transformers for this
!pip install git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08
!pip install datasets==2.10.1
import locale # colab workaround
locale.getpreferredencoding = lambda: "UTF-8" # colab workaround
!pip install wandb

  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-uuvne3tv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-uuvne3tv
  Resolved https://github.com/huggingface/transformers.git to commit b873234cb649a24865021f0d598627ce2b24d34a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 36.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
 

In [1]:
from datetime import datetime
import os
import sys

import torch
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq

In [2]:
from datasets import load_dataset
dataset = load_dataset("b-mc2/sql-create-context", split="train")
train_dataset = dataset.train_test_split(test_size=0.1)["train"]
eval_dataset = dataset.train_test_split(test_size=0.1)["test"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/b-mc2___json/b-mc2--sql-create-context-b6e5c5a03fb67709/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


In [13]:
train_dataset[0]

{'question': 'Name the average Played with a Points of 0*?',
 'context': 'CREATE TABLE table_name_72 (played INTEGER, points VARCHAR)',
 'answer': 'SELECT AVG(played) FROM table_name_72 WHERE points = "0*"'}

In [17]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [18]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_errors.py:304 in                  │
│ hf_raise_for_status                                                                              │
│                                                                                                  │
│   301 │   </Tip>                                                                                 │
│   302 │   """                                                                                    │
│   303 │   try:                                                                                   │
│ ❱ 304 │   │   response.raise_for_status()                                                        │
│   305 │   except HTTPError as e:                                                                 │
│   306 │   │   error_code = response.headers.get("X-Error-Code")                                  │
│   307 │   │   error_message = response.headers.get("X-Error-Message")                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/requests/models.py:1021 in raise_for_status              │
│                                                                                                  │
│   1018 │   │   │   )                                                                             │
│   1019 │   │                                                                                     │
│   1020 │   │   if http_error_msg:                                                                │
│ ❱ 1021 │   │   │   raise HTTPError(http_error_msg, response=self)                                │
│   1022 │                                                                                         │
│   1023 │   def close(self):                                                                      │
│   1024 │   │   """Releases the connection back to the pool. Once this method has been            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
HTTPError: 401 Client Error: Unauthorized for url: 
https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:402 in cached_file             │
│                                                                                                  │
│    399 │   user_agent = http_user_agent(user_agent)                                              │
│    400 │   try:                                                                                  │
│    401 │   │   # Load from URL or cache if already cached                                        │
│ ❱  402 │   │   resolved_file = hf_hub_download(                                                  │
│    403 │   │   │   path_or_repo_id,                                                              │
│    404 │   │   │   filename,                                                                     │
│    405 │   │   │   subfolder=None if len(subfolder) == 0 else subfolder,                         │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py:114 in _inner_fn    │
│                                                                                                  │
│   111 │   │   if check_use_auth_token:                                                           │
│   112 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   113 │   │                                           

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_errors.py:304 in                  │
│ hf_raise_for_status                                                                              │
│                                                                                                  │
│   301 │   </Tip>                                                                                 │
│   302 │   """                                                                                    │
│   303 │   try:                                                                                   │
│ ❱ 304 │   │   response.raise_for_status()                                                        │
│   305 │   except HTTPError as e:                                                                 │
│   306 │   │   error_code = response.headers.get("X-Error-Code")                                  │
│   307 │   │   error_message = response.headers.get("X-Error-Message")                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/requests/models.py:1021 in raise_for_status              │
│                                                                                                  │
│   1018 │   │   │   )                                                                             │
│   1019 │   │                                                                                     │
│   1020 │   │   if http_error_msg:                                                                │
│ ❱ 1021 │   │   │   raise HTTPError(http_error_msg, response=self)                                │
│   1022 │                                                                                         │
│   1023 │   def close(self):                                                                      │
│   1024 │   │   """Releases the connection back to the pool. Once this method has been            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
HTTPError: 401 Client Error: Unauthorized for url: 
https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:402 in cached_file             │
│                                                                                                  │
│    399 │   user_agent = http_user_agent(user_agent)                                              │
│    400 │   try:                                                                                  │
│    401 │   │   # Load from URL or cache if already cached                                        │
│ ❱  402 │   │   resolved_file = hf_hub_download(                                                  │
│    403 │   │   │   path_or_repo_id,                                                              │
│    404 │   │   │   filename,                                                                     │
│    405 │   │   │   subfolder=None if len(subfolder) == 0 else subfolder,                         │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py:114 in _inner_fn    │
│                                                                                                  │
│   111 │   │   if check_use_auth_token:                                                           │
│   112 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   113 │   │                                           